In [2]:
import image_processing #import the image processing script

dire_train = 'data/train/' #trainig diretory
dire_test = 'data/test/' #test diretory

Using TensorFlow backend.


# Reading the test and training images!

In [3]:
sizeImg = 200 #image dimension
images = image_processing.ImageProcessing() #Instantiating an object from class (ImageProcessing)

x_train, y_train = images.loadImages(dire_train,sizeImg) #Trainig image processing 
x_test, y_test = images.loadImages(dire_test,sizeImg) #Test imge processing

100%|██████████| 625/625 [00:05<00:00, 114.12it/s]


# Creating the model of convolutional neural network

In [5]:
"""Importing the Keras library for model building"""
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [6]:
"""Setting Network Hyperparameters"""
EPOCHS = 20 #Epocha number
INIT_LR = 1e-3 #Learning rate
BS = 40 # batch size, amount of images that will pass through the network in each season
opt = Adam(lr=INIT_LR, decay=INIT_LR / 100) #optimizer

In [7]:
model = Sequential() # Sequential model keras
#add model layers
"""Convolutional layers"""
model.add(Conv2D(64, kernel_size = 3, activation = 'relu', input_shape=(sizeImg,sizeImg,1))) #
model.add(Conv2D(32, kernel_size = 3, activation = 'relu'))
model.add(Conv2D(16, kernel_size = 3, activation = 'relu'))
model.add(Flatten())
"""Dense layers"""
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))# the last layer must have the same number of classes
model.compile(optimizer = opt, loss='categorical_crossentropy',metrics=['accuracy']) #Compiling the model by defining the optimizer the loss function and the evaluation metric
model.summary()#Print CNN architecture 

W0919 15:02:54.266713 139754306234176 deprecation_wrapper.py:119] From /home/bruno/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0919 15:02:54.272680 139754306234176 deprecation_wrapper.py:119] From /home/bruno/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0919 15:02:54.276598 139754306234176 deprecation_wrapper.py:119] From /home/bruno/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0919 15:02:54.373131 139754306234176 deprecation_wrapper.py:119] From /home/bruno/anaconda3/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0919 15:02:

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 198, 198, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 196, 196, 32)      18464     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 194, 194, 16)      4624      
_________________________________________________________________
flatten_1 (Flatten)          (None, 602176)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                9634832   
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 18        
Total para

# Training CNN

In [8]:
"""Save model treined"""
filepath='model.h5'#name model save
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')#salve best model
callbacks_list = [checkpoint]

"""here a keras technique is used to improve training accuracy, the images are molded for the network to learn better"""
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

"""CNN trained by passing training and test data jutamnete with hyperparameters"""
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1, callbacks = callbacks_list)

W0919 15:03:09.056821 139754306234176 deprecation.py:323] From /home/bruno/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0919 15:03:09.135034 139754306234176 deprecation_wrapper.py:119] From /home/bruno/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/20
127/127 [==============================] - 37s 291ms/step - loss: 0.4589 - acc: 0.7909 - val_loss: 0.3222 - val_acc: 0.8671

Epoch 00001: val_acc improved from -inf to 0.86705, saving model to model.h5
Epoch 2/20
127/127 [==============================] - 30s 235ms/step - loss: 0.3221 - acc: 0.8669 - val_loss: 0.5728 - val_acc: 0.7245

Epoch 00002: val_acc did not improve from 0.86705
Epoch 3/20
127/127 [==============================] - 29s 230ms/step - loss: 0.2916 - acc: 0.8738 - val_loss: 0.3027 - val_acc: 0.8796

Epoch 00003: val_acc improved from 0.86705 to 0.87963, saving model to model.h5
Epoch 4/20
127/127 [==============================] - 29s 230ms/step - loss: 0.2639 - acc: 0.8947 - val_loss: 0.2651 - val_acc: 0.8983

Epoch 00004: val_acc improved from 0.87963 to 0.89832, saving model to model.h5
Epoch 5/20
127/127 [==============================] - 29s 230ms/step - loss: 0.2543 - acc: 0.8965 - val_loss: 0.2566 - val_acc: 0.8969

Epoch 00005: val_acc did not impr